In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-d/final_preprocessed.csv


In [2]:
path_link="/kaggle/input/preprocessed-d/final_preprocessed.csv"
df=pd.read_csv(path_link)
df=df.sample(20000)

In [3]:
df['review_text'].fillna('', inplace=True)

In [4]:
X=df['review_text']
y=df['rating']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
logreg = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
svm = SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42)
voting_clf = VotingClassifier(estimators=[('logreg', logreg), ('svm', svm)], voting='soft')
voting_clf.fit(X_train_tfidf, y_train)

# Step 7: Make predictions on the test set
y_pred = voting_clf.predict(X_test_tfidf)
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score: {:.2f}".format(f1))